In [15]:
!awk '/^[>;]/ { if (seq) { print seq }; seq=""; print } /^[^>;]/ { seq = seq $0 } END { print seq }' ../../A.fasta > ../../A.tmp 
#!sed 's/ /_/g;N;s/\n/,/g;s/|/,/g;s/>//g' > ../../B.csv
!sed -i 's/ /_/g;N;s/\n/,/g;s/|/,/g' ../../A.tmp
!awk -F , 'NF == 10' <../../A.tmp > ../../A.csv

In [1]:
import numpy as np
import pandas as pd
import sys
import re
import csv
import collections as co
import itertools as it
import umap
import hdbscan
import time 
import scipy.spatial.distance as ssd
#from memory_profiler import memory_usage
from plotnine.data import *
from plotnine import *
%matplotlib inline

In [3]:
class vectorizer(object):
    
    def __init__(self, k = 7, convert = 0):
    
        self.k = k
        self.convert = convert
        self.exist = co.defaultdict(int) 
        self.keys = list(self.exist.keys())
        self.col = len(self.keys)
        self.row = 0
        self.matrix = np.empty((self.row, self.col, ),dtype = "float32")
        self.amino = co.defaultdict(str, {
            'AAA':'K', 'AAC':'N', 'AAG':'K', 'AAT':'N',
            'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
            'AGA':'R', 'AGC':'S', 'AGG':'R', 'AGT':'S',
            'ATA':'I', 'ATC':'I', 'ATG':'M', 'ATT':'I',
            'CAA':'Q', 'CAC':'H', 'CAG':'Q', 'CAT':'H',
            'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
            'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
            'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
            'GAA':'E', 'GAC':'D', 'GAG':'E', 'GAT':'D',
            'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
            'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
            'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',    
            'TAA':'Y', 'TAC':'*', 'TAG':'*', 'TAT':'Y',
            'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
            'TGA':'*', 'TGC':'C', 'TGG':'W', 'TGT':'C',
            'TTA':'L', 'TTC':'F', 'TTG':'L', 'TTT':'F'
        })
                
    def translate(self, read):
    
        chain = ''

        for i in range(len(read) - 2):
            trip = read[i:i+3]
            chain += self.amino[trip]

        return(chain)
    
    
    def adjust_to_data(self, infile):
    
        self.row = infile.shape[0]
            
        for line, read in infile.itertuples(index=True, name=None):

            if self.convert == 1:
                seq = self.translate(read)
                del read

                num = len(seq) - self.k + 1

                for i in range(num):
                    kmer = seq[i:i+self.k]
                    self.exist[kmer] = 0

            else:
                seq = read
                del read

                num = len(seq) - self.k + 1

                if re.match('^[ACGT]*$', seq): 
                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        self.exist[kmer] = 0
                else:
                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        if re.match('^[ACGT]*$', kmer): 
                            self.exist[kmer] = 0
            
        self.keys = list(self.exist.keys())
        self.col = len(self.keys)
        self.matrix = np.empty((self.row, self.col, ), dtype="float32")
        
        del seq
    
    def calculate_frequence(self, infile):
        
        for line, read in infile.itertuples(index=True, name=None): 
                 
            if self.convert == 1:
                seq = self.translate(read)
                del read

                counts = self.exist.copy()
                num = len(seq) - self.k + 1

                for i in range(num):
                    kmer = seq[i:i+self.k]
                    counts[kmer] += 1

            else:
                seq = read
                del read

                counts = self.exist.copy()
                num = len(seq) - self.k + 1

                if re.match('^[ACGT]*$', seq): 
                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        counts[kmer] += 1
                else:
                    for i in range(num):
                        kmer = seq[i:i+self.k]
                        if re.match('^[ACGT]*$', kmer): 
                            counts[kmer] += 1

            vector = np.array(list(counts.values()), dtype = "float32")/num

            self.matrix[line] = vector
            
            counts.clear()
            del vector
            del seq
            del counts
    
    
    def get_keys(self):
        
        return(self.keys)
    
    
    def get_matrix(self):
        
        return(self.matrix)

In [38]:
def main():

    print("Read input and settings file.", end = ' ')

    infile = '../../A.csv'   
    setfile = 'Input/settings.csv'
    outpath = 'Output/'
    mode = 0
    
    #checkup for mode setting formation etc.
    
    settings = pd.read_csv(setfile, sep = ',', na_filter = False, index_col = 0)
    upload = pd.read_csv(infile, sep = ',', na_filter = False, header = None)
    upload.columns = ['accession', 'strain', 'segment', 'protein', 'genus', 'subtype', 'date', 'host', 'curation', 'genome']
    upload.query('curation == "Pass"', inplace = True)
    segments = settings.index.values.tolist()
    clusters = co.defaultdict(list)
    exec_time = 0
    increment = 0

    print("Finished.")

    for segment in segments:
    
        print(f"Starting calculations for segment {segment}:")

        start_clust = time.perf_counter()
        parameter = settings.loc[segment].to_list()
        setting = [para if type(para) == str else para.item() for para in parameter]
        
        subset = upload.query('segment == @segment').reset_index()

        sequence = subset[['genome']].copy()
        accession = subset[['accession']].copy()
        subtype = subset[['subtype']].copy()

        if mode == 0 or mode == 2:
        
            print("- Nucleotide k-mer frequency calculation.", end = ' ')

            freq_nt = vectorizer(k = setting[0], convert = 0)
            freq_nt.adjust_to_data(sequence)
            freq_nt.calculate_frequence(sequence)

            matrix_nt = freq_nt.get_matrix()
            keys_nt = freq_nt.get_keys()

            del freq_nt

            print("Finished.")

            print("- Running UMAP for dimension reduction.", end = ' ')

            matrix_nt_red = umap.UMAP(
                n_neighbors = setting[1],
                min_dist = setting[2],
                n_components = setting[3],
                random_state = setting[4],
                metric = setting[5],
            ).fit_transform(matrix_nt)

            del matrix_nt

            print("Finished.")

        if mode == 1 or mode == 2:
        
            print("- Aminoacid k-mer frequency calculation.", end = ' ')

            freq_aa = vectorizer(k = setting[6], convert = 1)
            freq_aa.adjust_to_data(sequence)
            freq_aa.calculate_frequence(sequence)

            matrix_aa = freq_aa.get_matrix()
            keys_aa = freq_aa.get_keys()

            del freq_aa

            print("Finished.")

            print("- Running UMAP for dimension reduction.", end = ' ')

            matrix_aa_red = umap.UMAP(
                n_neighbors = setting[7],
                min_dist = setting[8],
                n_components = setting[9],
                random_state = setting[10],
                metric = setting[11],
            ).fit_transform(matrix_aa)

            del matrix_aa

            print("Finished.")

        if mode == 0:
        
            matrix = pd.concat([accession, pd.DataFrame(matrix_nt_red)], axis=1, copy = False, ignore_index = False).set_index('accession')

        elif mode == 1:
            
            matrix = pd.concat([accession, pd.DataFrame(matrix_aa_red)], axis=1, copy = False, ignore_index = False).set_index('accession')
            
        elif mode == 2:
            
            matrix = pd.concat([accession, pd.DataFrame(matrix_aa_red), pd.DataFrame(matrix_nt_red)], axis=1, copy = False, ignore_index = False).set_index('accession')
            
        else:
            exit("Wrong clustering mode.")
        
        print("- Running HDBscan for clustering.", end = ' ')

        matrix_clust = hdbscan.HDBSCAN(
            min_samples = setting[12], #larger the value the more conservative the clustering (more points will be declared as noise)
            min_cluster_size = setting[13], #minimum size that can become a cluster
            cluster_selection_epsilon = setting[14], #don't seperate clusters with a distance less than value
            alpha = setting[15], #don't mess with this
        ).fit(matrix)

        print("Finished.")

        print("- Centroid extraction.", end = ' ')

        clusterlabel = matrix_clust.labels_

        blank = pd.DataFrame(zip(clusterlabel, ['False'] * len(clusterlabel)), columns = ['cluster', 'centroid'])
        clusters[segment] = pd.concat([blank, subtype, accession], axis=1, copy = False).set_index('accession')

        num = clusters[segment]['cluster'].max()+1
        values = ['True']*num
        accessions = []
        exclude = []
        include = []
        overall_mean=0
        subs = co.defaultdict(list)

        for i in range(num):

            query = clusters[segment].query('cluster == @i')
            match = query.index.values.tolist()
            sub = matrix.filter(items = match, axis=0)
            dist = ssd.cdist(sub, sub, metric = setting[16])
            inner_mean = pd.DataFrame(dist, columns = match, index = match, dtype = 'float32').mean()
            accessions.append(inner_mean.idxmin())
            overall_mean = overall_mean + inner_mean.mean()

            for sub in query['subtype'].tolist():
                if re.match('^[H][0-9]+N[0-9]+$', sub): 
                    subs['H'].append(re.search('[H][0-9]+', sub).group(0))
                    subs['N'].append(re.search('[N][0-9]+', sub).group(0))
                else:
                    subs['X'].append('X0')
                    subs['X'].append('X0')

            if len(set(subs['H'])) == 1 and len(set(subs['N'])) == 1:
                exclude.append(2)
                if 'X' not in subs.keys():
                    include.append(2)
            elif len(set(subs['H'])) == 1:
                exclude.append(1)
                if 'X' not in subs.keys():
                    include.append(1)
            elif len(set(subs['N'])) == 1:
                exclude.append(0)
                if 'X' not in subs.keys():
                    include.append(0)

            subs.clear()

        centroids = pd.DataFrame(values, columns=['centroid'], index = accessions)

        clusters[segment].loc[clusters[segment]['cluster'] != -1, ['cluster']] = clusters[segment].loc[clusters[segment]['cluster'] != -1, ['cluster']] + increment
        clusters[segment].update(centroids)

        increment += num - 1

        print("Finished.")

        stop_clust = time.perf_counter()
        exec_clust = stop_clust - start_clust
        exec_time = exec_time + exec_clust

        print(f"- Clustering done in {exec_clust:0.2f} seconds.")
        diagnostic = co.Counter(clusterlabel)
        print(f"- {str(len(clusterlabel))} sequences, {str(diagnostic[-1])} unclustered, {str(len(set(diagnostic))-1)} cluster.")
        print(f"- Mean of inner cluster distance mean {overall_mean/num:0.10f}")
        print(f"- {exclude.count(0) + exclude.count(2)}({include.count(0) + include.count(2)}) clusters containing matching NA types.")
        print(f"- {exclude.count(1) + exclude.count(2)}({include.count(1) + include.count(2)}) clusters containing matching HA types.")
        print("Finished.")

    result = pd.concat(clusters)
    result.index.set_names(["segment", "accession"], inplace=True)
    result.reset_index(level = "segment", inplace=True)  
    result.sort_values(by=['segment', 'cluster', 'subtype'], inplace = True)
    result.to_csv(outpath + 'cluster_wo_align.csv', index=True, header=True, sep=',', mode='w')
    
    density = pd.DataFrame(result.value_counts(subset=['segment', 'cluster']), columns = ['size'])
    density.reset_index(level = ["segment", "cluster"], inplace=True)
    
    plot = (ggplot(density, aes(x="size", colour = "factor(segment)", fill = "factor(segment)")) 
     + labs(
        x="Cluster Size",
        y="Density",
        fill="Segment",
        colour="Segment",
        title="Cluster Size Distribution",
     )
     + geom_density(alpha = 0.1)
     + scale_x_log10()
     + scale_y_continuous()
    # + scale_y_log10()
    )
    plot.save(outpath + 'density.csv')
    
    print(f"Overall execution time {exec_time:0.2f} seconds.")

In [39]:
if __name__ == "__main__":

    main()
    #memory = memory_usage(main)
    #print(f"Maximum memory used: {max(memory)/1000:0.2f} Gb.")

Read input and settings file. Finished.
Starting calculations for segment 1:
- Nucleotide k-mer frequency calculation. Finished.
- Running UMAP for dimension reduction. Finished.
- Running HDBscan for clustering. Finished.
- Centroid extraction. Finished.
- Clustering done in 687.20 seconds.
- 55436 sequences, 0 unclustered, 441 cluster.
- Mean of inner cluster distance mean 0.0001269798
- 259(236) clusters containing matching NA types.
- 269(243) clusters containing matching HA types.
Finished.
Starting calculations for segment 2:
- Nucleotide k-mer frequency calculation. Finished.
- Running UMAP for dimension reduction. Finished.
- Running HDBscan for clustering. Finished.
- Centroid extraction. Finished.
- Clustering done in 744.98 seconds.
- 55292 sequences, 1 unclustered, 435 cluster.
- Mean of inner cluster distance mean 0.0001319285
- 286(264) clusters containing matching NA types.
- 291(270) clusters containing matching HA types.
Finished.
Starting calculations for segment 3:
-